In [137]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas
import csv
import numpy

In [231]:
#Select Data
train_data = pandas.read_csv('train.csv')
test_data = pandas.read_csv('test.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [232]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [233]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [234]:
#fill in missing values for Age, Cabin, and Embarked (and Fare for test data) and encode Sex to a binary variable
#for age and fare set it equal to mean age, for Cabin set binary of whether they have a cabin or not, for embarked, set to the mode
#create variable to see if passenger is traveling alone
train_X = train_data.drop(['Survived'], axis = 1)
train_X["Age"][train_X["Age"].isna()] = train_X["Age"].mean()
train_X["Embarked"].fillna(train_X['Embarked'].mode()[0], inplace = True)
for i in train_X.index:
    train_X.at[i, 'Sex'] = 0 if train_X.at[i,'Sex']=='male' else 1
    train_X.at[i,'Cabin'] = 0 if pandas.isnull(train_X.at[i,'Cabin']) else 1
train_X['Travel Alone'] = numpy.where(train_X['SibSp'] + train_X['Parch'] >0 , 0, 1)
train_Y = train_data['Survived']
test_X = test_data
test_X["Age"][test_X["Age"].isna()] = test_X["Age"].mean()
for i in test_X.index:
    test_X.at[i, 'Sex'] = 0 if test_X.at[i,'Sex']=='male' else 1
    test_X.at[i,'Fare'] = test_X["Fare"].mean() if pandas.isnull(test_X.at[i,'Fare']) else test_X.at[i,'Fare']
    test_X.at[i,'Cabin'] = 0 if pandas.isnull(test_X.at[i,'Cabin']) else 1
test_X['Travel Alone'] = numpy.where(test_X['SibSp'] + test_X['Parch'] >0 , 0, 1)
print(train_X.isnull().sum(), test_X.isnull().sum())


c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


PassengerId     0
Pclass          0
Name            0
Sex             0
Age             0
SibSp           0
Parch           0
Ticket          0
Fare            0
Cabin           0
Embarked        0
Travel Alone    0
dtype: int64 PassengerId     0
Pclass          0
Name            0
Sex             0
Age             0
SibSp           0
Parch           0
Ticket          0
Fare            0
Cabin           0
Embarked        0
Travel Alone    0
dtype: int64


In [235]:
#figure out important variables
survive = train_data.groupby("Survived").mean()
s1 = survive.iloc[0,:]
s2 = survive.iloc[1,:]
abs((s2-s1)/(s1+s2))

PassengerId    0.002971
Pclass         0.129755
Age            0.038706
SibSp          0.077914
Parch          0.170176
Fare           0.372661
dtype: float64

In [236]:
#drop unnecessary variables
train_X = train_X.drop(['SibSp', 'Parch', 'PassengerId', 'Ticket', 'Name'], axis =1)
test_X = test_X.drop(['SibSp', 'Parch', 'PassengerId', 'Ticket', 'Name'], axis = 1)
test_X.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,Travel Alone
0,3,0,34.5,7.8292,0,Q,1
1,3,1,47.0,7.0000,0,S,0
2,2,0,62.0,9.6875,0,Q,1
3,3,0,27.0,8.6625,0,S,1
4,3,1,22.0,12.2875,0,S,0


In [237]:
#create dummy variables for Pclass and Embarked
train_X = pandas.get_dummies(train_X, columns = ["Pclass", "Embarked"])
test_X = pandas.get_dummies(test_X, columns = ["Pclass", "Embarked"])

In [238]:
#create and fit Logistic Regression object
logisticRegression = LogisticRegression()
fittedReg = logisticRegression.fit(train_X,train_Y)

c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [239]:
prediction = fittedReg.predict(test_X)

In [240]:
results = []
for i in test_data.index:
    individual = [test_data.at[i,'PassengerId'],prediction[i]]
    results.append(individual)


In [241]:
#check accuracy on a split of the train set
X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, test_size =.2, random_state = 42)


In [242]:
testFit = logisticRegression.fit(X_train, y_train)
pred = testFit.predict(X_test)
accuracy_score(y_test, pred)

c:\users\adamw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8044692737430168

In [244]:
with open('titanic_results.csv', 'w', newline = '') as file:
    writer = csv.DictWriter(file, fieldnames = ["PassengerId","Survived"])
    writer.writeheader()
    for i in results:
        writer.writerow({"PassengerId": i[0], "Survived":i[1]})